<a href="https://colab.research.google.com/github/yoant1/explorasi_hard_skill_DE_python_programming/blob/main/tokopedia_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# --- Instalasi Library Python yang Dibutuhkan ---
# Selenium: Untuk otomatisasi browser dan web scraping
# Pandas: Untuk manipulasi dan analisis data dalam bentuk DataFrame
# Matplotlib & Seaborn: Untuk membuat visualisasi (grafik)
# BeautifulSoup4: Meskipun tidak menjadi fokus utama dengan Selenium di sini,
#                  tetap baik untuk diinstal sebagai alat bantu parsing HTML
!pip install selenium pandas matplotlib seaborn beautifulsoup4

# --- Instalasi Chrome Browser dan ChromeDriver (Otomatis untuk Colab) ---
# apt-get: Perintah untuk menginstal paket di sistem Linux (yang digunakan Colab)
!apt-get update # Memperbarui daftar paket yang tersedia
!apt-get install -y chromium-browser # Menginstal browser Chromium (versi open-source dari Chrome)
!pip install chromedriver-autoinstaller # Menginstal library Python untuk mendownload ChromeDriver yang cocok secara otomatis

# --- Import Library Python yang Akan Digunakan ---
# Ini harus diimpor SETELAH instalasi selesai
import chromedriver_autoinstaller # Untuk instalasi ChromeDriver otomatis
from selenium import webdriver # Modul utama Selenium
from selenium.webdriver.chrome.service import Service # Untuk konfigurasi service ChromeDriver
from selenium.webdriver.common.by import By # Untuk menentukan cara mencari elemen (misal: berdasarkan CSS Selector, ID, dll.)
from selenium.webdriver.support.ui import WebDriverWait # Untuk menunggu elemen muncul secara dinamis
from selenium.webdriver.support import expected_conditions as EC # Kondisi yang diharapkan saat menunggu
from selenium.common.exceptions import TimeoutException, NoSuchElementException # Untuk menangani error spesifik dari Selenium
import time # Untuk memberikan jeda waktu
import re # Untuk ekspresi reguler (membersihkan teks)
import pandas as pd # Untuk DataFrame
import matplotlib.pyplot as plt # Untuk plotting grafik dasar
import seaborn as sns # Untuk visualisasi statistik yang lebih indah

# --- Konfigurasi dan Inisialisasi WebDriver ---
# chromedriver_autoinstaller akan secara otomatis mencari dan menginstal ChromeDriver yang sesuai
chromedriver_autoinstaller.install()

# Opsi untuk menjalankan Chrome dalam mode 'headless' (tanpa antarmuka grafis)
# Ini penting karena Google Colab adalah lingkungan server tanpa GUI
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Mode headless: browser berjalan di latar belakang
options.add_argument('--no-sandbox') # Diperlukan untuk menjalankan Chrome di lingkungan virtual seperti Colab
options.add_argument('--disable-dev-shm-usage') # Mencegah masalah memori di lingkungan Colab

# Inisialisasi WebDriver
try:
    driver = webdriver.Chrome(options=options)
    print("STATUS: WebDriver berhasil diinisialisasi untuk Google Colab (headless).")
except Exception as e:
    print(f"ERROR: Terjadi kesalahan saat menginisialisasi WebDriver: {e}")
    print("Pastikan instalasi Chrome Browser dan ChromeDriver berhasil di atas, dan opsi Chrome sudah benar.")
    print("Jika masalah berlanjut, coba restart runtime Colab (Runtime -> Restart runtime) dan jalankan lagi semua sel dari awal.")
    exit() # Keluar dari program jika WebDriver tidak bisa diinisialisasi


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [4]:
# --- Konfigurasi URL dan Variabel Scraping ---
# Ganti 'laptop' dengan kata kunci pencarian lain jika diinginkan
base_url = "https://www.tokopedia.com/search?st=product&q=laptop"
all_product_data = [] # List untuk menyimpan semua data produk yang berhasil di-scrape
num_pages_to_scrape = 3 # Jumlah halaman yang ingin kita scrape

print(f"\n--- Memulai Proses Web Scraping dari {num_pages_to_scrape} Halaman Tokopedia ---")

# --- Loop untuk Melakukan Scraping pada Setiap Halaman ---
for page_num in range(1, num_pages_to_scrape + 1):
    url = f"{base_url}&page={page_num}" # Membangun URL untuk halaman yang sedang diproses
    driver.get(url) # Membuka URL di browser Selenium
    print(f"\nSTATUS: Mengakses halaman: {url}")

    # --- Penanganan Konten Dinamis (Menunggu & Scroll) ---
    try:
        # Tunggu hingga elemen kartu produk (product-card) muncul di halaman
        # Ini penting agar JavaScript memiliki waktu untuk memuat konten
        print(f"DEBUG: Menunggu elemen produk muncul di halaman {page_num}...")
        WebDriverWait(driver, 20).until( # Tunggu maksimal 20 detik
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-testid="product-card"]'))
        )
        print(f"DEBUG: Elemen 'product-card' berhasil ditemukan.")

        # Lakukan scroll ke bawah berulang kali untuk memuat semua produk (lazy loading)
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) # Beri jeda 2 detik agar konten baru sempat dimuat
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print(f"DEBUG: Scroll selesai di halaman {page_num}. Tidak ada lagi konten baru yang dimuat.")
                break # Berhenti jika tinggi halaman tidak bertambah (berarti semua konten sudah dimuat)
            last_height = new_height
        print(f"STATUS: Konten halaman {page_num} berhasil dimuat sepenuhnya (setelah scroll).")

    except TimeoutException:
        print(f"ERROR: Timeout! Produk tidak ditemukan di halaman {page_num} setelah 20 detik.")
        print(f"DEBUG: Panjang source code halaman saat timeout: {len(driver.page_source)} karakter.")
        # Simpan screenshot untuk debugging visual jika terjadi timeout
        driver.save_screenshot(f'/content/timeout_page_{page_num}.png')
        print(f"DEBUG: Screenshot halaman {page_num} disimpan sebagai /content/timeout_page_{page_num}.png")
        continue # Lanjutkan ke halaman berikutnya jika halaman ini gagal dimuat

    # --- Ekstraksi Data Produk dari Halaman Saat Ini ---
    products = driver.find_elements(By.CSS_SELECTOR, '[data-testid="product-card"]')
    print(f"DEBUG: Ditemukan total {len(products)} elemen produk di halaman {page_num} untuk diekstrak.")

    if not products:
        print(f"WARNING: Tidak ada elemen produk yang valid ditemukan di halaman {page_num} untuk diekstrak. Mungkin selector salah atau situs memblokir akses.")
        continue # Lanjut ke halaman berikutnya

    for product in products:
        try:
            # Ekstraksi Nama Produk
            name_element = product.find_element(By.CSS_SELECTOR, '[data-testid="lblPDPDetailProductName"]')
            name = name_element.text.strip() if name_element else "N/A"

            # Ekstraksi Harga Produk
            price_element = product.find_element(By.CSS_SELECTOR, '[data-testid="lblPDPDetailProductPrice"]')
            price_text = price_element.text.strip() if price_element else "N/A"
            # Membersihkan teks harga (menghilangkan 'Rp' dan '.') lalu mengkonversi ke float
            price = float(re.sub(r'[Rp.]', '', price_text)) if price_text != "N/A" else 0.0

            # Ekstraksi Rating Produk
            rating_element = product.find_elements(By.CSS_SELECTOR, '[data-testid="rating"]')
            rating = 0.0
            if rating_element:
                # Contoh: aria-label="Rating 4.5 dari 5" -> kita ambil angka 4.5
                match = re.search(r'Rating (\d+\.?\d*)', rating_element[0].get_attribute('aria-label'))
                if match:
                    rating = float(match.group(1))

            # Ekstraksi Jumlah Ulasan Produk (mungkin ada dua selector yang berbeda)
            reviews = 0
            reviews_element_1 = product.find_elements(By.CSS_SELECTOR, '.css-1bidzud span.css-1rn6k3f')
            if reviews_element_1 and "Ulasan" in reviews_element_1[0].text:
                match = re.search(r'(\d+)\s*Ulasan', reviews_element_1[0].text)
                if match:
                    reviews = int(match.group(1))
            else: # Coba selector alternatif jika yang pertama tidak ditemukan
                reviews_element_2 = product.find_elements(By.CSS_SELECTOR, '[data-testid="lblCardRatingProductCount"]')
                if reviews_element_2:
                    match = re.search(r'(\d+)', reviews_element_2[0].text)
                    if match:
                        reviews = int(match.group(1))

            # Tambahkan data produk ke list
            all_product_data.append({
                'Nama Produk': name,
                'Harga': price,
                'Rating': rating,
                'Jumlah Ulasan': reviews
            })

        except NoSuchElementException:
            # DEBUG: print(f"DEBUG: Beberapa elemen tidak ditemukan untuk satu produk di halaman {page_num}. Melewati produk ini.")
            continue # Lanjutkan ke produk berikutnya jika ada elemen yang tidak ditemukan
        except Exception as e:
            # DEBUG: print(f"DEBUG: Error umum saat mengekstrak data produk di halaman {page_num}: {e}. Melewati produk ini.")
            continue # Lanjutkan ke produk berikutnya jika ada error lainnya

    print(f"STATUS: Selesai mengekstrak data dari halaman {page_num}.")
    print(f"STATUS: Total produk terkumpul hingga saat ini: {len(all_product_data)}")
    time.sleep(2) # Beri jeda 2 detik antar halaman untuk mengurangi risiko deteksi bot

# Tutup browser Selenium setelah semua proses scraping selesai
driver.quit()
print("\n--- Proses Scraping Selesai. Browser Ditutup. ---")

# --- Langkah 3: Tampilkan Detail dalam Pandas DataFrame dan Visualisasi ---
print("\n--- Memulai Analisis dan Visualisasi Data ---")

# Buat DataFrame dari data yang terkumpul
df = pd.DataFrame(all_product_data)

# --- Pemeriksaan dan Ringkasan DataFrame ---
if not df.empty:
    print("\n--- DataFrame Berhasil Dibuat ---")
    print("5 Baris Pertama Data:")
    print(df.head())

    print("\nInformasi Umum DataFrame (Tipe Data, Non-Null Count):")
    df.info()

    print("\nStatistik Deskriptif (Rata-rata, Min, Max, dll. untuk kolom numerik):")
    print(df.describe())

    # --- Pembersihan Tipe Data untuk Visualisasi ---
    # Pastikan kolom numerik memiliki tipe data yang benar, 'coerce' akan mengubah error menjadi NaN
    df['Harga'] = pd.to_numeric(df['Harga'], errors='coerce')
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
    df['Jumlah Ulasan'] = pd.to_numeric(df['Jumlah Ulasan'], errors='coerce')

    # Hapus baris yang memiliki nilai kosong (NaN) di kolom kunci setelah konversi
    # Ini penting agar visualisasi tidak error karena data tidak valid
    df.dropna(subset=['Harga', 'Rating', 'Jumlah Ulasan'], inplace=True)

    if df.empty: # Cek lagi setelah dropna
        print("\nWARNING: DataFrame kosong setelah proses pembersihan data. Tidak ada data yang valid untuk divisualisasikan.")
    else:
        print("\n--- Memulai Pembuatan Visualisasi ---")

        # 1. Distribusi Harga Produk
        plt.figure(figsize=(12, 7))
        sns.histplot(df['Harga'], bins=50, kde=True, color='skyblue')
        plt.title('Distribusi Harga Produk Laptop di Tokopedia', fontsize=16)
        plt.xlabel('Harga (Rp)', fontsize=12)
        plt.ylabel('Frekuensi', fontsize=12)
        plt.ticklabel_format(style='plain', axis='x') # Mematikan notasi ilmiah pada sumbu X
        plt.grid(axis='y', alpha=0.75)
        plt.tight_layout()
        plt.show()

        # 2. Distribusi Rating Produk
        plt.figure(figsize=(10, 6))
        sns.countplot(x=df['Rating'].round(1), data=df, palette='viridis', order=sorted(df['Rating'].round(1).unique())) # Mengelompokkan rating ke 1 desimal
        plt.title('Distribusi Rating Produk Laptop di Tokopedia', fontsize=16)
        plt.xlabel('Rating (Skala 0-5)', fontsize=12)
        plt.ylabel('Jumlah Produk', fontsize=12)
        plt.grid(axis='y', alpha=0.75)
        plt.tight_layout()
        plt.show()

        # 3. Hubungan antara Harga dan Rating (dengan ukuran titik berdasarkan Jumlah Ulasan)
        plt.figure(figsize=(12, 7))
        # Hue dan size berdasarkan Jumlah Ulasan untuk menambah informasi visual
        sns.scatterplot(x='Harga', y='Rating', data=df, alpha=0.7, hue='Jumlah Ulasan', size='Jumlah Ulasan', sizes=(20, 400), palette='coolwarm')
        plt.title('Hubungan Harga dan Rating Produk Laptop (Ukuran Titik = Jumlah Ulasan)', fontsize=16)
        plt.xlabel('Harga (Rp)', fontsize=12)
        plt.ylabel('Rating', fontsize=12)
        plt.ticklabel_format(style='plain', axis='x')
        plt.grid(True, alpha=0.75)
        plt.legend(title='Jumlah Ulasan', bbox_to_anchor=(1.05, 1), loc='upper left') # Pindahkan legenda agar tidak menutupi plot
        plt.tight_layout()
        plt.show()

        # 4. Top 10 Produk dengan Ulasan Terbanyak (Jika ada data ulasan)
        if not df.empty and 'Jumlah Ulasan' in df.columns and df['Jumlah Ulasan'].sum() > 0:
            top_reviews_products = df.sort_values(by='Jumlah Ulasan', ascending=False).head(10)
            if not top_reviews_products.empty:
                plt.figure(figsize=(12, 8))
                sns.barplot(x='Jumlah Ulasan', y='Nama Produk', data=top_reviews_products, palette='rocket')
                plt.title('10 Produk Laptop dengan Ulasan Terbanyak', fontsize=16)
                plt.xlabel('Jumlah Ulasan', fontsize=12)
                plt.ylabel('Nama Produk', fontsize=12)
                plt.tight_layout()
                plt.show()
            else:
                print("DEBUG: Setelah sorting, top_reviews_products kosong.")
        else:
            print("INFO: Tidak ada data ulasan yang cukup untuk menampilkan Top 10 produk.")

else:
    print("\nERROR: DataFrame kosong. Tidak ada data yang berhasil di-scrape untuk ditampilkan atau divisualisasikan.")
    print("Silakan periksa output di atas untuk pesan ERROR/WARNING saat proses scraping untuk menemukan penyebabnya.")


--- Memulai Proses Web Scraping dari 3 Halaman Tokopedia ---

STATUS: Mengakses halaman: https://www.tokopedia.com/search?st=product&q=laptop&page=1
DEBUG: Menunggu elemen produk muncul di halaman 1...
ERROR: Timeout! Produk tidak ditemukan di halaman 1 setelah 20 detik.
DEBUG: Panjang source code halaman saat timeout: 180829 karakter.
DEBUG: Screenshot halaman 1 disimpan sebagai /content/timeout_page_1.png

STATUS: Mengakses halaman: https://www.tokopedia.com/search?st=product&q=laptop&page=2
DEBUG: Menunggu elemen produk muncul di halaman 2...
ERROR: Timeout! Produk tidak ditemukan di halaman 2 setelah 20 detik.
DEBUG: Panjang source code halaman saat timeout: 180829 karakter.
DEBUG: Screenshot halaman 2 disimpan sebagai /content/timeout_page_2.png

STATUS: Mengakses halaman: https://www.tokopedia.com/search?st=product&q=laptop&page=3
DEBUG: Menunggu elemen produk muncul di halaman 3...
ERROR: Timeout! Produk tidak ditemukan di halaman 3 setelah 20 detik.
DEBUG: Panjang source code 